In [1]:
import geodb.model
from geodb.model import GPSPoint, db_url, GPSTrack

In [2]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [3]:
engine = sqlalchemy.create_engine(db_url(), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
import pandas as pd
import numpy as np

In [5]:
query = "select * from track where source is null"
display(pd.read_sql_query(query, engine))

,id,name,comment,description,source,type,properties,raw
0,7241,None,None,None,None,train,{},None
1,7242,None,None,None,None,walking,{},None
2,7243,None,None,None,None,train,{},None
3,7244,None,None,None,None,stationary,{},None
4,7245,None,None,None,None,airplane,{},None
...,...,...,...,...,...,...,...,...
2498,7131,None,None,None,None,airplane,{},None
2499,7132,None,None,None,None,tram,{},None
2500,7133,None,None,None,None,walking,{},None
2501,7134,None,None,None,None,walking,{},None


In [6]:
track = session.query(GPSTrack).get(6987)

In [7]:
track.name

'Diamond Head'

In [8]:
from geojson import FeatureCollection

In [9]:
features = FeatureCollection([track.as_geojson_feature(None)])

import json
from decimal import Decimal

def default(obj):
    if isinstance(obj, Decimal):
        return str(obj)
    raise TypeError("Object of type '%s' is not JSON serializable" % type(obj).__name__)

geojson = json.dumps(features, default=default)


In [10]:
from IPython.display import GeoJSON

In [11]:
GeoJSON(features)

<IPython.display.GeoJSON object>

In [12]:
query = """
select type, count(*) from track
where id in (
  select distinct(track_id)
  from point
  where time >= '2019-06-25'
  and time < '2019-07-16')
group by 1
"""
display(pd.read_sql_query(query, engine))

,type,count
0,airplane,15
1,car,672
2,None,746
3,stationary,814
4,train,2
5,walking,532
6,Waypoint,380
7,bus,11
8,boat,1
9,cycling,26


In [13]:
safari_tracks = session.query(GPSTrack).filter(GPSTrack.id.in_(
    session.query(GPSPoint.track_id)
    .filter(GPSPoint.time > '2019-06-25')
    .filter(GPSPoint.time < '2019-07-16')
.distinct())).all()

In [14]:
len(safari_tracks)

3199

In [40]:
df = pd.DataFrame([[t, t.type, len(t.points)] + list(t.bounds) + [t.start_localtime, t.end_localtime] for t in safari_tracks],
                  columns=["t", "type", "points", "east", "north", "west", "south", "start", "end"])

In [57]:
np.min(df['east']), np.max(df['west']), np.min(df['north']), np.max(df['south'])


(-347.252896698764, 350.13696524544764, -347.252896698764, 350.13696524544764)

In [55]:
df[df.west < 180]

,t,type,points,east,north,west,south,start,end
0,<geodb.model.GPSTrack object at 0x7f92a6e87be0>,cycling,1,36.129331,-0.423684,36.129331,36.129331,2019-06-30 00:26:18+00:00,2019-06-30 00:26:18+00:00
1,<geodb.model.GPSTrack object at 0x7f92a6e87c88>,None,1,36.129369,-0.423695,36.129369,36.129369,2019-06-30 00:26:25+00:00,2019-06-30 00:26:25+00:00
2,<geodb.model.GPSTrack object at 0x7f92a6e87d30>,cycling,1,36.129480,-0.423727,36.129480,36.129480,2019-06-30 00:26:36+00:00,2019-06-30 00:26:36+00:00
3,<geodb.model.GPSTrack object at 0x7f92a6e87dd8>,None,1,36.129490,-0.423730,36.129490,36.129490,2019-06-30 00:26:39+00:00,2019-06-30 00:26:39+00:00
4,<geodb.model.GPSTrack object at 0x7f92a6e87e80>,cycling,178,36.128638,-0.423735,36.129524,36.129524,2019-06-30 00:26:43+00:00,2019-06-30 00:51:43+00:00
...,...,...,...,...,...,...,...,...,...
3194,<geodb.model.GPSTrack object at 0x7f92a6a34d30>,Waypoint,1,36.129486,-0.422580,36.129486,36.129486,2019-06-29 16:26:53+00:00,2019-06-29 16:26:53+00:00
3195,<geodb.model.GPSTrack object at 0x7f92a6a34dd8>,Waypoint,1,36.129308,-0.423747,36.129308,36.129308,2019-06-29 17:23:23+00:00,2019-06-29 17:23:23+00:00
3196,<geodb.model.GPSTrack object at 0x7f92a6a34e80>,Waypoint,1,36.129437,-0.423695,36.129437,36.129437,2019-06-29 17:38:48+00:00,2019-06-29 17:38:48+00:00
3197,<geodb.model.GPSTrack object at 0x7f92a6a34f28>,Waypoint,1,36.129382,-0.423710,36.129382,36.129382,2019-06-29 18:09:05+00:00,2019-06-29 18:09:05+00:00


In [49]:
GeoJSON(FeatureCollection([t.as_geojson_feature(True) for t in df[df.east < 0][df.north < 5]['t']]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


<IPython.display.GeoJSON object>

In [16]:
east_tracks = [t for t in safari_tracks 
               if t.points[0].corrected_coords(True)[0] < 180
              and t.points[-1].corrected_coords(True)[0] < 180]

In [17]:
safari_features = [t.as_geojson_feature(True) for t in east_tracks]

In [ ]:
safari = FeatureCollection(safari_features)

In [ ]:
GeoJSON(safari)

In [ ]:
pd.DataFrame([(t.name, t.type, t.points[0].corrected_coords()) for t in safari_tracks[0:10]])

In [ ]:
len(east_tracks)

In [ ]:
df = pd.DataFrame([t.points[0].corrected_coords(True) for t in east_tracks])

In [ ]:
np.min(df[0]), np.max(df[0])

In [ ]:
(1, 2, 3)[0:2]